# Code Interpreting with Fireworks LLM
This example uses the E2B's [Code Interpreter](https://github.com/e2b-dev/code-interpreter) as a tool for Firefunction-V2 by Fireworks. We ask the Firefunction-V2 LLM to **TBD TBD TBD**
>

First, we install Fireworks, OpenAI, and the E2B Code Interpreter SDK.

In [1]:
%pip install --upgrade fireworks-ai openai e2b_code_interpreter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 8.9 MB/s eta 0:00:00:00:01
  Attempting uninstall: openai
    Found existing installation: openai 1.12.0
    Uninstalling openai-1.12.0:
      Successfully uninstalled openai-1.12.0
  Attempting uninstall: e2b_code_interpreter
    Found existing installation: e2b-code-interpreter 0.0.8
    Uninstalling e2b-code-interpreter-0.0.8:
      Successfully uninstalled e2b-code-interpreter-0.0.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
crewai 0.1.32 requires langchain<0.2.0,>=0.1.0, but you have langchain 0.2.3 which is incompatible.
crewai 0.1.32 requires langchain-openai<0.0.3,>=0.0.2, but you have langchain-openai 0.1.8 which is incompatible.
open-interpreter 0.2.2 requires tiktoken<0.6.0,>=0.5.2, but you have tiktoken 0.7.0 

We import packages.

In [ ]:
import openai
import json

Get your API keys or save them to .env file.

TBD: Add E2B API Key, rename Fireworks API Key, add links.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

# TODO: Get your OpenAI API key
FIREWORKS_API_KEY = os.getenv("FIREWORKS_API_KEY")

# TODO: Get your E2B API key from https://e2b.dev/docs
E2B_API_KEY = os.getenv("E2B_API_KEY")

Then we set up the model to use.

We describe the functions available to the agent in great detail in JSON Schema. Here, it's just one function, the one that will execute the code generated by the LLM assistant.

**TBD**: Add the fireworks api key as the api_key variable to client.

In [ ]:
SYSTEM_PROMPT = """
## your job & context
you are a python data scientist. you are given tasks to complete and you run python code to solve them.
You DO NOT MAKE SYNTAX MISTAKES OR FORGET ANY IMPORTS
- the python code runs in jupyter notebook.
- every time you call `execute_python` tool, the python code is executed in a separate cell. it's okay to multiple calls to `execute_python`.
- display visualizations using matplotlib or any other visualization library directly in the notebook. don't worry about saving the visualizations to a file.
- you have access to the internet and can make api requests.
- you also have access to the filesystem and can read/write files.
- you can install any pip package (if it exists) if you need to but the usual packages for data analysis are already preinstalled.
- you can run any python code you want, everything is running in a secure sandbox environment.
"""


client = openai.OpenAI(
    base_url = "https://api.fireworks.ai/inference/v1",
    api_key = "<YOUR_FIREWORKS_API_KEY>"
)

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "What are Nike's net income in 2022?"}
]

tools = [
    {
        "type": "function",
        "function": {
            "name": "execute_python",
            "description": "Execute python code in a Jupyter notebook cell and returns any result, stdout, stderr, display_data, and error.",
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "The python code to execute in a single cell.",
                    },
                },
                "required": ["code"],
            },
        },
        },
]

chat_completion = client.chat.completions.create(
    model="accounts/fireworks/models/fw-function-call-34b-v0",
    messages=messages,
    tools=tools,
    temperature=0.1
)
print(chat_completion.choices[0].message.model_dump_json(indent=4))

In [ ]:
# Locally available tool implementation. Can be replaced by any remote API
def get_financial_data(metric: str, financial_year: int, company: str):
    print(f"{metric=} {financial_year=} {company=}")
    if metric == "net_income" and financial_year == 2022 and company == "Nike":
        return {"net_income": 6_046_000_000}
    else:
        raise NotImplementedError()

function_call = chat_completion.choices[0].message.tool_calls[0].function
tool_response = locals()[function_call.name](**json.loads(function_call.arguments))
print(tool_response)

In [ ]:
agent_response = chat_completion.choices[0].message

# Append the response from the agent
messages.append(
    {
        "role": agent_response.role, 
        "content": "",
        "tool_calls": [
            tool_call.model_dump()
            for tool_call in chat_completion.choices[0].message.tool_calls
        ]
    }
)

# Append the response from the tool 
messages.append(
    {
        "role": "function",
        "content": json.dumps(tool_response)
    }
)

next_chat_completion = client.chat.completions.create(
    model="accounts/fireworks/models/fw-function-call-34b-v0",
    messages=messages,
    tools=tools,
    temperature=0.1
)

print(next_chat_completion.choices[0].message.content)